# Get Embeddings For My Videos

In [ ]:
import openai

In [ ]:
openai.api_key = 'your api key goes here'

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('video_transcripts.csv')

In [ ]:
embeddings = []

In [ ]:
for txt in df['text']:
    embeddings.append(get_embedding(txt))

In [ ]:
df['embeddings'] = embeddings

In [ ]:
df.to_csv('vid_with_embeddings.csv')

In [ ]:
df.head()

In [ ]:
df = pd.read_csv('vid_with_embeddings.csv')


In [ ]:
embeddings = df['embeddings']

In [ ]:
new_embeddings = []
# convert the string representation of the lists to a list of floats
for e in embeddings:
    new_embeddings.append([float(x) for x in e[1:-1].split(',')])

In [ ]:
df['embeddings'] = new_embeddings
print(df.head())

In [ ]:
print(len(df['embeddings'][0]))

In [ ]:
df.to_csv("final_embeddings.csv")

In [ ]:
df.shape

In [ ]:
!pip install pinecone-client

In [ ]:
import pinecone
pinecone.init(api_key="your api key goes here", environment='us-west1-gcp')

In [ ]:
index = pinecone.Index("your index name goes here")

# Upserting Data Into Pinecone
- We'll have the text associated with it's embedding, and the url and start time passed in as metadata

In [ ]:
# upserting a list of tuples of the form (text, embedding,  {"link": link, "start": start})
vals = []
for i in range(len(df)):
    vals.append((df['text'][i], df['embeddings'][i], {"link": df['video_link'][i], "start": df['start_time'][i]}))

In [ ]:
print(vals[0])

In [ ]:
for i in range(0, len(vals), 100):
    try:
        index.upsert(vals[i:i+100])
    except:
        print("error")
    print(f"{i/len(vals)*100:.2f}%")